In [ ]:
import os
import sys

sys.path.insert(0, os.path.abspath("../utils"))
from aoc_utils import load_data, check

In [ ]:
import heapq

In [ ]:
data = load_data(2024, 16)

In [ ]:
# data, part_1, part_2
tests = [
    (
        """###############
#.......#....E#
#.#.###.#.###.#
#.....#.#...#.#
#.###.#####.#.#
#.#.#.......#.#
#.#.#####.###.#
#...........#.#
###.#.#####.#.#
#...#.....#.#.#
#.#.#.###.#.#.#
#.....#...#.#.#
#.###.#.#.#.#.#
#S..#.....#...#
###############
""",
        7036,
        45,
    ),
    (
        """#################
#...#...#...#..E#
#.#.#.#.#.#.#.#.#
#.#.#.#...#...#.#
#.#.#.#.###.#.#.#
#...#.#.#.....#.#
#.#.#.#.#.#####.#
#.#...#.#.#.....#
#.#.#####.#.###.#
#.#.#.......#...#
#.#.###.#####.###
#.#.#...#.....#.#
#.#.#.#####.###.#
#.#.#.........#.#
#.#.#.#########.#
#S#.............#
#################
""",
        11048,
        64,
    ),
]

# Part 1

In [ ]:
def get_layout(data):
    walls = set()
    for j, line in enumerate(data.splitlines()):
        for i, c in enumerate(line):
            if c == "#":
                walls.add((i, j))
            elif c == "S":
                start = i, j
            elif c == "E":
                finish = i, j
            else:
                assert c == "."
    return walls, start, finish

In [ ]:
def lowest_path_score(walls, start, finish, direction=(1, 0)):
    lowest_scores = {}
    queue = [(0, (*start, *direction))]
    while queue:
        s, pos = heapq.heappop(queue)
        if pos in lowest_scores and lowest_scores[pos] <= s:
            continue
        lowest_scores[pos] = s
        i, j, di, dj = pos
        if (i, j) == finish:
            return s
        if (i, j) in walls:
            continue
        heapq.heappush(queue, (s + 1, (i + di, j + dj, di, dj)))
        heapq.heappush(queue, (s + 1000, (i, j, -dj, di)))
        heapq.heappush(queue, (s + 1000, (i, j, dj, -di)))
    raise ValueError("Unreachable finish")

In [ ]:
def reindeer_race(data, score=lowest_path_score):
    return score(*get_layout(data))

In [ ]:
check(reindeer_race, tests)
reindeer_race(data)

# Part 2

In [ ]:
def best_paths_length(walls, start, finish, direction=(1, 0)):
    score_with_prev = {}
    queue = [(0, (*start, *direction), None)]
    best = None
    ends = set()
    while queue:
        s, pos, prev = heapq.heappop(queue)
        if best is not None and s > best:
            # compute best paths
            res = {finish}
            while ends:
                prev = ends.pop()
                res.add(prev[:2])
                ends |= score_with_prev[prev][1]
            return len(res)
        i, j, di, dj = pos
        if (i, j) in walls:
            continue
        if (i, j) == finish:
            ends.add(prev)
            best = s
        if pos in score_with_prev:
            ps, pprev = score_with_prev[pos]
            assert ps <= s
            if ps == s:
                score_with_prev[pos] = (s, pprev | {prev})
            continue
        score_with_prev[pos] = (s, set() if prev is None else {prev})
        heapq.heappush(queue, (s + 1, (i + di, j + dj, di, dj), pos))
        heapq.heappush(queue, (s + 1000, (i, j, -dj, di), pos))
        heapq.heappush(queue, (s + 1000, (i, j, dj, -di), pos))
    raise ValueError("Unreachable finish")

In [ ]:
check(reindeer_race, tests, 2, score=best_paths_length)
reindeer_race(data, score=best_paths_length)